# Recurrent Neural Network
기존 neural network에서는 뉴런에 입력 벡터가 들어가면, parameter들과의 계산 후(예: Wx+b) activation function을 지나 출력되었다. <br />
RNN에서는 새로운 가중치로 '과거의 자신'이 추가된다. 따라서 은닉층의 메모리 셀은 tanh(Wx+W'h+b)와 같이 계산된다. <br />
이때 Wx는 기존 뉴런처럼 입력 벡터와 그를 위한 가중치를 나타내고, W'h는 이전 메모리 셀의 값 h와 그를 위한 가중치 W'를 나타낸다.<br />
h' = tanh(Wx+W'h+b) 로 계산된 값은 출력층에서 또 f(W''h'+b)같이 계산되어 출력된다.<br />



## 1. 은닉 셀 구현하기

In [ ]:
import numpy as np

timesteps = 10      # 시점의 수. input vector들이 입력되는 횟수이다.
input_size = 4      # input vector의 size
hidden_size = 8     # 은닉층의 메모리 셀의 용량

inputs = np.random.random( (timesteps, input_size) )   # 랜덤으로, 입력될 값 10개 (각각 크기 4) 생성 
hidden_state_t = np.zeros((hidden_size,))              # 각 은닉층의 메모리 셀은 0으로 초기화

Wx = np.random.random( (hidden_size, input_size) )     # 입력 벡터에 대한 가중치 (8,4)
Wh = np.random.random( (hidden_size, hidden_size) )     # 은닉 상태(이전 t의 자신)에 대한 가중치 (8,8)
b = np.random.random( (hidden_size,)) 

In [ ]:
total_hidden_states = []

for input_t in inputs :
  output_t = np.tanh(np.dot(Wx, input_t)+np.dot(Wh, hidden_state_t)+b)
  total_hidden_states.append(list(output_t))
  hidden_state_t = output_t

print(np.stack(total_hidden_states, axis=0))

[[0.95303276 0.90343227 0.88426935 0.94407579 0.95485008 0.9252148
  0.87426494 0.9785663 ]
 [0.99987321 0.99983392 0.99999654 0.99999751 0.99999214 0.99991579
  0.99995054 0.99999042]
 [0.99993685 0.99956887 0.99999522 0.99999701 0.99999001 0.99993005
  0.99994002 0.99999203]
 [0.9999174  0.9999021  0.99999894 0.99999863 0.99999676 0.99997041
  0.99998745 0.99999731]
 [0.99996926 0.99997711 0.99999967 0.99999958 0.99999904 0.99998845
  0.99999531 0.9999992 ]
 [0.9998565  0.99972533 0.99999709 0.99999716 0.99999242 0.99997541
  0.99998534 0.99999837]
 [0.99991267 0.99978131 0.99999755 0.99999786 0.99999391 0.99995364
  0.99997264 0.99999558]
 [0.99991645 0.99993062 0.99999932 0.9999988  0.99999754 0.99998092
  0.99999313 0.99999838]
 [0.99994763 0.99993819 0.99999937 0.999999   0.99999785 0.99998237
  0.99999264 0.99999844]
 [0.99996132 0.99992886 0.99999849 0.99999935 0.99999758 0.99996982
  0.99997817 0.99999818]]


##2. nn.RNN()으로 구현하기

In [1]:
import torch
import torch.nn as nn

#### 은닉층 1개

In [10]:
input_size = 5
hidden_size = 8

inputs = torch.Tensor(1, 10, 5)     # 배치 1개, 시점 10개, 각각 크기 5
cell = nn.RNN(input_size, hidden_size, batch_first=True)    # batch_first=True는 input의 첫 성분이 배치 크기라는 뜻
outputs, _status = cell(inputs)     # output: 모든 시점의 은닉값들, _status: 최종 시점의 은닉값
                                    # output: (1, 10, 8)            _status: (1, 1, 8)

#### 은닉층 여러개

In [16]:
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=4, batch_first=True)     # 4개의 은닉층
outputs, _status = cell(inputs)

print(outputs.shape)    # 모든 시점의 hidden state들. 근데 마지막 층의 hidden state만 출력된다. 따라서 (1, 10, 8)로 동일
print(_status.shape)    # 마지막 시점의 hidden state. layer가 4개이니 (4, 1, 8)이 된다.

torch.Size([1, 10, 8])
torch.Size([4, 1, 8])


##3. Bidirectional Recurrent Neural Network
한 timestep마다 메모리 셀을 두 개 사용한다. <br />
하나는 RNN과 동일하게 이전 시점의 hidden state를 받아오고, 나머지 하나는 이후 시점의 hidden state를 받아온다.<br />
그리고 출력은 이 두 셀을 모두 사용해서 이루어진다.<br />
layer가 늘어난다면, 아래 layer의 두 셀과 위 layer의 두 셀이 모두 얽혀서 전달되게 된다.

In [19]:
inputs = torch.Tensor(1, 10, 5)     # 배치 1개, 시점 10개, 각각 size 5
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=3, batch_first=True, bidirectional=True)    # bidirectional=True를 추가하면 양방향 RNN이 된다.
outputs, _status = cell(inputs)

print(outputs.shape)    # 모든 시점, 마지막 layer의 hidden state이다. 따라서 (1, 10, 16)이다.
                        # 10은 시점 개수, 16은 hidden_size의 두 배이다. 메모리 셀이 두 개씩 있고 그 두개를 concatenate한 게 출력된다

print(_status.shape)    # 마지막 시점, 모든 layer의 hidden state이다. 따라서 (6, 1, 8))이다.
                        # 6은 layer 개수 3의 두배이다. (메모리 셀이 2개)  timestep은 하나니까 1, 그리고 각 hidden state의 size는 8.

torch.Size([1, 10, 16])
torch.Size([6, 1, 8])


# LSTM(Long Short-Term Memory)
<br />
RNN의 한계는 timestep이 진행될 수록 이전 input들이 잊혀진다는 것이다. 즉 RNN은 기억력이 부족하다고 할 수 있다.<br />
이를 장기 의존성 문제 라고 한다. 그리고 이것을 해결할 방법이 LSTM이다.<br />

## 내부 구조
1. 입력 게이트<br />
  현재 정보를 기억하기 위한 게이트이다.<br />
  1-1. 현재 시점의 input과 이전 시점의 hidden state에 각각 가중치를 곱해서 __$tanh$__ 의 activation function에 통과시킨 것이 $g_t$가 된다.<br />
  1-2. 현재 시점의 input과 이전 시점의 hidden state에 각각 가중치를 곱해서 __$\sigma$__의 activation function에 통과시킨 것이 $i_t$가 된다.<br /><br />
2. 삭제 게이트<br />
  현재 시점의 input과 이전 시점의 hidden state에 각각 가중치를 곱해서 __$\sigma$__를 통과하는데, 결과로 0~1의 값이 나온다.<br />
  이 결과를 $f_t$라고 하고 이 값이 0에 가까울 수록 '기억'을 많이 지우게 된다.<br /><br />
3. 셀 상태(장기 상태)<br />
  3-1. 이전 셀 상태 $C_{t-1}$과 $f_t$의 entrywise 곱을 한다.<br />
  3-2. $g_t$와 $i_t$도 entrywise 곱을 한다.<br />
  3-3. 이 두 값을 더한 것이 t시점의 셀 상태 $C_t$가 된다.<br />
  삭제 게이트의 값에 따라 과거의 값에 얼마나 의존할지 결정된다.<br /><br />
4. 출력 게이트<br />
  4-1. 현재 시점의 input과 이전 시점의 hidden state에 각각 가중치를 곱해서 __$\sigma$__를 통과시킨다. 이를 $o_t$라고 한다.<br />
  4-2. $o_t$와 $tanh(c_t)$의 entrywise 곱이 현재 시점의 은닉값(단기 상태)가 된다. 즉 t시점의 hidden state $h_t$가 되고, 이는 다음 timestep으로<br />
전달되고, 다음 layer(혹은 출력층)로도 전달되게 된다.




Pytorch에서는 간단하게 구현할 수 있다.

In [4]:
input_dim=5
hidden_size=8

cell = nn.LSTM(input_dim, hidden_size, batch_first=True)  

# RNN으로 간단한 모델 구현하기
문자 단위 RNN

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [92]:
input_str = 'recurrent'
label_str = 'neuralnet'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print(vocab_size)

8


In [93]:
input_size = vocab_size  
hidden_size = 8
output_size = vocab_size
learning_rate=0.1

In [94]:
char_to_index = dict((c,i) for i, c in enumerate(char_vocab))

index_to_char = {}
for key, value in char_to_index.items() :
  index_to_char[value] = key


x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]

x_data = [x_data]   # 배치 차원 추가
y_data = [y_data]   # 배치 차원 추가

x_one_hot = [np.eye(vocab_size)[x] for x in x_data] # one-hot encoding

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))    # 9개 문자가, 12차원 one-hot encode되었기 때문에 (1,9,12)
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 9, 8])
레이블의 크기 : torch.Size([1, 9])


In [95]:
class Net(torch.nn.Module) :
  def __init__(self, input_size, hidden_size, output_size) :
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)  # 출력층으로 전결합층 사용

  def forward(self, x) :
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [98]:
net = Net(input_size, hidden_size, output_size)

outputs = net(X)
print(outputs.shape)   # (1, 9, 9)   배치 1개, 시점은 9개(input string이 9문자니까), output size 9.

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate)

for epoch in range(101) :
  optimizer.zero_grad()
  output = net(X)
  loss = criterion(output.view(-1, input_size), Y.view(-1))     # batch 차원을 없애고 input_size 차원과 합친다.
  loss.backward()
  optimizer.step()

  if epoch%20==0 :
    result = output.data.numpy().argmax(axis=2)
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(epoch, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

torch.Size([1, 9, 8])
0 loss:  2.090449333190918 prediction:  [[5 2 2 2 2 5 2 2 5]] true Y:  [[4, 2, 7, 5, 0, 3, 4, 2, 6]] prediction str:  reeeereer
20 loss:  0.013998348265886307 prediction:  [[4 2 7 5 0 3 4 2 6]] true Y:  [[4, 2, 7, 5, 0, 3, 4, 2, 6]] prediction str:  neuralnet
40 loss:  0.0016270828200504184 prediction:  [[4 2 7 5 0 3 4 2 6]] true Y:  [[4, 2, 7, 5, 0, 3, 4, 2, 6]] prediction str:  neuralnet
60 loss:  0.0009611963760107756 prediction:  [[4 2 7 5 0 3 4 2 6]] true Y:  [[4, 2, 7, 5, 0, 3, 4, 2, 6]] prediction str:  neuralnet
80 loss:  0.0007754646358080208 prediction:  [[4 2 7 5 0 3 4 2 6]] true Y:  [[4, 2, 7, 5, 0, 3, 4, 2, 6]] prediction str:  neuralnet
100 loss:  0.0006671748124063015 prediction:  [[4 2 7 5 0 3 4 2 6]] true Y:  [[4, 2, 7, 5, 0, 3, 4, 2, 6]] prediction str:  neuralnet
